### Using the awk commands for parsing the log file

In [1]:
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt

from shutil import rmtree

root_path = os.path.join(Path().resolve().parent.parent)
package_path = os.path.join(root_path, "hamageolib")
test_fixture_path = os.path.join(root_path, "big_tests", "TwoDSubduction", "eba_cdpt_coh500_SA80.0_cd7.5_log_ss300.0")
result_path = os.path.join(Path().resolve(), "results")

if str(package_path) not in sys.path:
    sys.path.insert(0, str(package_path))

if not os.path.isdir(result_path):
    os.mkdir(result_path)

from utils.case_options_haoyuan import parse_log_file_for_time_info_to_pd


#### Parsing Log File for Time Step and Wall Clock Information

This block extracts and processes time step and wall clock data from an ASPECT log file.

- Define the path to the log file: $ \text{log\_file\_path} $ within the test fixture directory.
- Define the output file path: $ \text{output\_path} $ for storing parsed results.
- Use the function `parse_log_file_for_time_info_to_pd()` to extract time-related information into a pandas DataFrame.
- The resulting pandas data structure contains a "Corrected Wall Clock" as output for wall clocks corrected for restarting the case.

In [2]:
log_file_path = os.path.join(test_fixture_path, "output", "log.txt")
output_path = os.path.join(result_path, "parse_time_info_results")

time_dat = parse_log_file_for_time_info_to_pd(log_file_path)

#### Use the CASE_OPTIONS class to load a case

In [3]:
from utils.case_options_haoyuan import CASE_OPTIONS

Case_Options = CASE_OPTIONS(test_fixture_path)

# print(Case_Options.time_df)

### Use the CASE_SUMMARY class to generate summary of a project

This code loops through **case directories** inside `"big_tests/TwoDSubduction"` and updates the case summary. 

#### **To locate a case:**
- Each valid case directory **must contain a `.prm` file**.
- The `.prm` file is identified using `find_case_files(item_path)`.
- If no `.prm` file is found, the directory is skipped.
- The summary stores essential metadata about cases.

#### **Output:**
- The script updates the df pandas object as an attribute of the class
- For example, `Case_Summary.df["basename"]` stores processed case names.
- The `Case_Summary.df.columns` stores all the available names.
- The `Case_Summary.df.attrs["units"]` stores all the units.
- The final output prints the collected case information.

In [ ]:
from utils.case_options_haoyuan import CASE_SUMMARY, find_case_files

# Initiate the class
Case_Summary = CASE_SUMMARY()

# Assign a directory to loop over
test_fixture_parent_dir = os.path.join(root_path, "big_tests", "TwoDSubduction")

# find case directory and update summary
for item in os.listdir(test_fixture_parent_dir):
    item_path = os.path.join(test_fixture_parent_dir, item)

    if os.path.isdir(item_path):
        prm_file, _ = find_case_files(item_path)
        
        if prm_file is not None:
            Case_Summary.update_single_case(item_path)


print(str(Case_Summary.df.columns))
print(str(Case_Summary.df.attrs["units"]))

#### Convert to a latex table

- A `column_names` could be added to select the columns to export. Otherwise all columns are exported.

In [ ]:
from utils.case_options_haoyuan import df_to_latex

output_path = os.path.join(result_path, "case_options_results.tex")

columnn_names = ["name", "end time step", "end time"]

df_to_latex(Case_Summary.df, output_path, columnn_names)